## Import

In [1]:
from wmb import cemba
from ALLCools.mcds import MCDS

import pandas as pd
from ALLCools.clustering import log_scale

In [2]:
var_dim = 'geneslop2k-vm23'
chrom_to_remove = ['chrX', 'chrY', 'chrM', 'chrL']
downsample = 1000
mc_type = 'CHN'
std_cutoff = 0.05

In [3]:
# Parameters
chrom_to_remove = ["chrX", "chrY", "chrM", "chrL"]
cpu = 1
downsample = 1000
group_name = "CorticalExc"
mc_type = "CHN"
mem_gb = 1
std_cutoff = 0.05
var_dim = "geneslop2k-vm23"


## Select cells

In [4]:
cells = pd.read_csv('mc_cells.txt', index_col=0, header=None).index
cells.name = 'cell'
cells.size

123904

## Get adata with CEF

In [5]:
mcds = MCDS.open(cemba.CEMBA_SNMC_MCDS_PATH, var_dim=var_dim, use_obs=cells)

mcds

<xarray.MCDS>
Dimensions:                   (cell: 123904, count_type: 2, geneslop2k-vm23: 32208, mc_type: 2)
Coordinates:
  * cell                      (cell) <U15 '10A_M_1181' ... '9F_M_2348'
  * count_type                (count_type) <U3 'mc' 'cov'
  * geneslop2k-vm23           (geneslop2k-vm23) <U18 'ENSMUSG00000051951' ......
    geneslop2k-vm23_chrom     (geneslop2k-vm23) <U5 'chr1' 'chr1' ... 'chrM'
    geneslop2k-vm23_cov_mean  (geneslop2k-vm23) float64 6.313e+03 ... 312.2
    geneslop2k-vm23_end       (geneslop2k-vm23) int64 3673498 3515553 ... 16299
    geneslop2k-vm23_start     (geneslop2k-vm23) int64 3203901 3464587 ... 12145
  * mc_type                   (mc_type) <U3 'CGN' 'CHN'
Data variables:
    geneslop2k-vm23_da        (cell, geneslop2k-vm23, mc_type, count_type) uint16 dask.array<chunksize=(1000, 32208, 1, 1), meta=np.ndarray>
    geneslop2k-vm23_da_frac   (cell, geneslop2k-vm23, mc_type) float32 dask.array<chunksize=(1000, 32208, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  geneslop2k-vm23

In [6]:
mc_cef = pd.read_csv('mC.CEF.csv', header=None, index_col=0).squeeze()
rna_cef = pd.read_csv('RNA.CEF.csv', header=None, index_col=0).squeeze()
cef = mc_cef | rna_cef
cef = cef[cef].index
cef.name = var_dim

In [7]:
mcds.add_feature_selection_column(cef, f'{var_dim}_cef')
adata = mcds.get_adata(mc_type=mc_type,
                       select_hvf=f'{var_dim}_cef',
                       split_large_chunks=False)

In [8]:
std_filter = adata.X.std(axis=0) > std_cutoff
adata._inplace_subset_var(std_filter)

In [9]:
adata

AnnData object with n_obs × n_vars = 123904 × 10842
    var: 'chrom', 'cov_mean', 'end', 'start', 'cef'

## Preprocessing and save

In [10]:
# log mC fraction and scale features
log_scale(adata, with_mean=True)

# reverse mC fraction so its positively corr with RNA
adata.X *= -1

In [11]:
adata.write_h5ad('mc_input.h5ad')